### Determining the most effective variant caller with machine learning

**Background**

* There is substantial disagreement between various variant calling pipelines (i.e.: a tools that tells a user whether a variant exists within a particular site within the genome)

* There have been many efforts to resolve the discrepancy between vairant callers, but an effective way to resolve these issues has yet to be identified

* Crowdsourcing efforts have been launched to more confidently identify variants within the human genome

* For the following, I will use a machine learning classifier - Random Forest Classifier - to determine the accuracy of the genotype calls generated by a varaint caller built based on Illumina technology

***
Overview
***
1. Train the classifier using crowdsource data
2. Determine the accuracy of the classifier
3. Determine how well the predictions made by the classfier correlate with the calls made by the variant caller

#### Data Summary
Structural varaints from a Personal Genome Project genome (Ashkenazi Jewish son) were assessed using an Illumnia sequencing and variant calling pipeline. 

* 1516 Instances (Variants)
* 43 Features (Generated via the variant calling pipeline)

A study led by Peyton Greenside (Stanford) and Google Verily used crowdsourcing to assign genotypes to each of the 1516 variants. These genotypes will be compared to the genotypes generated by the variant calling pipeline. The crowdsourced gentoypes are considered 'ground truth' and will be used in the initial training/testing of the model.

##### Labels

**Crowdsource Data**

| Label | Definition           |
|-------|----------------------|
|   0   | Homozygous Variant   |
|   1   | Heterozygous Variant |
|   2   | Homozygous Reference |


**Variant Caller Data**

| Label | Definition           |
|-------|----------------------|
|   0   | Homozygous Reference |
|   1   | Heterozygous Variant |
|   2   | Homozygous Variant   |
|   -1  | Unknown              |





Crowdsource data source: http://biorxiv.org/content/early/2016/12/13/093526

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import pandas as pd
from sklearn.feature_selection import RFECV
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import f1_score
from sklearn import preprocessing
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE


/Users/lmc2/anaconda/envs/trees/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/lmc2/anaconda/envs/trees/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
df = pd.read_csv('/Users/lmc2/Desktop/NIHFAES/FinalProject/Train/Data/CrowdVar.Train_250bp_HG002.csv') 
X = pd.read_csv('/Users/lmc2/Desktop/NIHFAES/FinalProject/Train/Data/CrowdVar.Train_250bp_HG002.csv')
X.drop(["sample", "chrom", "CN0_prob", "CN1_prob", "CN2_prob", "GTcons", "GTconflict", "GTsupp"], axis=1, inplace=True)
X = X.dropna()
X2 = X.dropna()
Y = X.pop('Label')

X3 = pd.DataFrame()
X3 = X
# X.drop(["sample"], axis=1, inplace=True)
X.head()

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.7, random_state=0)

In [4]:
model = RandomForestClassifier() 
#out of bag samples to estimate general accuracy
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [15]:
pred = model.predict_proba(X)

In [6]:
X_test['pred'] = pred

ValueError: Wrong number of items passed 3, placement implies 1

model.feature_importances_

In [1]:
%matplotlib inline
feature_importances = pd.Series(model.feature_importances_, index=X.columns)
feature_importances.sort()
feature_importances.plot.bar()

NameError: name 'pd' is not defined

In [8]:
predict_label = model.predict(X_test)
predict_label

array([1, 0, 0, ..., 1, 0, 1])

In [9]:
X_test['predicted_labels'] = predict_label
X_test['true_labels'] = y_test


/Users/lmc2/anaconda/envs/trees/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/lmc2/anaconda/envs/trees/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [10]:
X_test.head()

,start,end,Ill250.alt_alnScore_mean,Ill250.alt_alnScore_std,Ill250.alt_count,Ill250.alt_insertSize_mean,Ill250.alt_insertSize_std,Ill250.alt_reason_alignmentScore,Ill250.alt_reason_insertSizeScore,Ill250.alt_reason_orientation,...,Ill250.ref_count,Ill250.ref_insertSize_mean,Ill250.ref_insertSize_std,Ill250.ref_reason_alignmentScore,Ill250.ref_reason_orientation,Ill250.GT,size,Svsize,predicted_labels,true_labels
1222,37002875,37002995,933.818182,33.460474,11,424.181818,72.344904,10,1,0,...,36,448.500000,97.328910,36,0,1,120,120,1,1
310,53594099,53595428,912.200000,40.041978,5,381.200000,67.957045,5,0,0,...,0,0.000000,0.000000,0,0,-1,1329,1329,0,0
9,8698627,8698845,975.320000,14.981909,50,441.260000,83.787782,50,0,0,...,0,0.000000,0.000000,0,0,2,218,218,0,0
785,32153720,32153877,977.625000,19.877358,16,451.750000,98.450178,15,1,0,...,31,438.548387,109.845886,31,0,1,157,157,1,1
295,3885212,3885330,967.558823,17.692124,34,429.558824,77.334716,34,0,0,...,0,0.000000,0.000000,0,0,2,118,118,0,0


In [21]:
predictions = model.predict_proba(X)
predictions

array([[ 1. ,  0. ,  0. ],
       [ 1. ,  0. ,  0. ],
       [ 1. ,  0. ,  0. ],
       ..., 
       [ 0.1,  0.9,  0. ],
       [ 0. ,  1. ,  0. ],
       [ 0.1,  0.9,  0. ]])

#### Compare predicted datapoints to datapoints produced by the caller

In [51]:
y_true = X_test['Ill250.GT']
y_pred = X_test['predicted_labels']
y_prob = X_test['']
f1_score(y_true, y_pred, average=None)

/Users/lmc2/anaconda/envs/trees/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


array([ 0.        ,  0.03278689,  0.9201278 ,  0.        ])

### Accuracy Metrics
[for the overall model]

In [44]:
score = metrics.accuracy_score(y_test, model.predict(X_test))
score

0.96889726672950049

#### Future Directions

- Create a scatter plot to display groups and highlight datapoints were predict_prob are low (ie: outliers)

- use k-fold cross validation to train the model

- Retrain the model with the most important features